In [1]:
from glob import glob
import os
import cv2
import csv
import numpy as np

# read files

In [2]:
csv_file = "dataset/valid.csv"
#csv_file = "dataset/valid.csv"

In [3]:
f = open(csv_file, 'r', encoding="utf-8")
rdr = csv.reader(f)
filename = []
features = []
for line in rdr:
    #print(line)
    #print("len(line): ", len(line))
    try:
        filename.append(line[0])
        features.append(line[1:])
    except:
        pass
f.close()

In [4]:
len(filename)

10600

In [5]:
len(features)

10600

# get positive features

In [6]:
features_ps = []

In [7]:
features = features[1:]

In [8]:
for row in features:
    #print(len(row))
    tup_list = []
    for i in range(0,len(row)-1,2):
        #print(i)
        
        tup = ( float(row[i]), float(row[i+1]) )
        if tup[0] != -1:
            tup_list.append(tup)
        
    features_ps.append(tup_list)

In [9]:
features_x = []
features_y = []
for item in features_ps:
    #item => list of tuples
#     print(item)
    temp_x = [] 
    temp_y = []
    for tup in item:
#         print("tup[0]: ", tup[0])
#         print("tup[1]:", tup[1])
        temp_x.append(tup[0])
        temp_y.append(tup[1])
    features_x.append(temp_x)
    features_y.append(temp_y)

# print(features_x)
# print(features_y)

In [10]:
print(features_x[0])
print(min(features_x[0]))
print(max(features_x[0]))

[404.0, 467.0, 494.0, 428.0, 428.0, 516.0, 517.0, 426.0, 458.0, 497.0, 442.0, 442.0, 513.0, 513.0, 472.0, 513.0, 504.0, 469.0, 506.0, 506.0, 506.0, 507.0]
404.0
517.0


# get min, max of features

In [11]:
features_minmax = []
for i in range(0, len(features_ps)):
    assert len(features_x[i]) == len(features_y[i])
    
#     print("len(features_x[i]):", len(features_x[i]))
#     print("len(features_y[i]):", len(features_y[i]))
    
#     if len(features_x[i]) != 0:
#         print("i: ", i)
#         print("xmin: ", min(features_x[i]))
#         print("ymin: ", min(features_y[i]))
#         print("xmax: ", max(features_x[i]))
#         print("ymax: ", max(features_y[i]))
#         print("---")
#         # [xmin, ymin, xmax, ymax]
    
    if len(features_x[i]) == 0 or len(features_y[i]) ==0:
        features_minmax.append([0, 0, 0, 0])
    else:
        features_minmax.append([min(features_x[i]), min(features_y[i]), max(features_x[i]), max(features_y[i])])

In [12]:
print(len(features_minmax))

10599


In [13]:
for item in features_minmax:
    xmin, ymin, xmax, ymax = item
    assert xmin <= xmax
    assert ymin <= ymax

# landmark, bbox 그리기

In [14]:
len(features_x)

10599

In [15]:
len(features_ps)

10599

In [16]:
len(features_y)

10599

In [17]:
img = cv2.imread("dataset/infiniq_sample.jpg")

h, w, c = img.shape

In [18]:
print(h, w, c)

480 853 3


In [19]:
bbox = []
for index in range(len(features_ps)):
#     img = cv2.imread(item, cv2.IMREAD_COLOR)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     h, w , c = img.shape
#     index = filename.index(os.path.basename(item))
    
    #print("index: ", index)
    xmin, ymin, xmax, ymax = features_minmax[index]
    
    
    if xmin==0 and ymin==0 and xmax==0 and ymax == 0:
        bbox.append([0,0,0,0])
    else:
        
        x_mean = int(np.array(features_x[index]).mean())
        y_mean = int(np.array(features_y[index]).mean())
        x_std = int(np.array(features_x[index]).std())
        y_std = int(np.array(features_y[index]).std())

        xmin = xmin - x_std
        ymin = ymin - y_std
        xmax = xmax + x_std
        ymax = ymax + y_std

        if xmin <= 0:
            xmin = 0
        if ymin <= 0:
            ymin = 0        
        if xmax >= w:
            xmax = w
        if ymax >= h:
            ymax = h
            
            
        #assert xmax>=xmin
        #assert ymax>=ymin
    
    #     print("index: ", index)    
    #     print("xmin: ", xmin)
    #     print("ymin: ", ymin)
    #     print("xmax: ", xmax)
    #     print("ymax: ", ymax)
    #     print("---")
    
        bbox.append([xmin, ymin, xmax, ymax])
    #     # draw landmark
#     for tup in int_features[index]:
#         if tup[0] == -1:
#             pass
#         cv2.circle(img, tup, 4, (0,0,255), -1)
    
#     # draw bbox
#     cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0,255,0), 3 )
#     save_path = "./dataset/20200902_day_shadow_01_Class 1. 정상주행/frame/0_20200902_day_shadow_01/output/"+ os.path.basename(item) +".jpg"
#     cv2.imwrite(save_path, img)

In [20]:
len(bbox)

10599

In [21]:
filename = filename[1:]


In [22]:
len(filename)

10599

In [23]:
len(features_x)

10599

In [24]:
len(features_ps)

10599

In [25]:
len(features_y)

10599

In [26]:
len(features)

10599

In [27]:
mode="train"

f = open("dataset/train_valid_infiniq.txt", "w")
for i in range(len(features)):
    xmin, ymin, xmax, ymax = bbox[i]
    
    x = xmin
    y = ymin
    w = xmax - xmin
    h = ymax - ymin

    if xmin==0 and ymin==0 and xmax ==0 and ymax ==0:
        print("[skipped - b/c zero] i: ", i)
        continue
    if x < 0 or y < 0 or w < 0 or h < 0:
        print("[skipped - b/c neg] i: ", i)
        continue
    
    f.write("# "+filename[i]+"\n")
    
    #print("i: ", i)
    assert x >= 0
    assert y >= 0
    assert w >= 0
    assert h >= 0
    
    b = " ".join([str(x),str(y),str(w),str(h)])
    
    temp = [features[i][inner_i] +" " +features[i][inner_i+1] for inner_i in range(0, len(features[0])-1, 2)]
    ft = " 0 ".join(temp)
    
    
    if mode=="train": f.write(b + " " + ft + " 0 1"+ "\n")
    if mode=="valid": f.write(b + "\n")
    
f.close()

[skipped - b/c neg] i:  2299
[skipped - b/c neg] i:  2300
[skipped - b/c neg] i:  2302
[skipped - b/c neg] i:  2303
[skipped - b/c zero] i:  2590
[skipped - b/c neg] i:  2608
[skipped - b/c zero] i:  8116
